In [3]:
import fitz  # PyMuPDF
import os

# Define the base directory where the original PDF is located and where the new directory will be created
base_directory = 'D:\\Python Practice files'
pdf_path = os.path.join(base_directory, 'Transcripts of bachelors degree.pdf')

# Create a new directory for the individual PDF files
new_directory_name = 'Individual pdf files for Transcripts'
new_directory_path = os.path.join(base_directory, new_directory_name)
os.makedirs(new_directory_path, exist_ok=True)

doc = fitz.open(pdf_path)

output_definitions = [
    {"name": "Bachelor Degree Transcript", "pages": [0]},
    {"name": "Bachelor Degree Certificate", "pages": [1]},
    {"name": "Info", "pages": [2]},
    {"name": "Credential Evaluation", "pages": list(range(3, len(doc)))}
]

for output_def in output_definitions:
    output_doc = fitz.open()
    for page_num in output_def["pages"]:
        output_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
    
    # Adjust the output path to save in the new directory
    output_file_name = f"{output_def['name'].replace(' ', '_')}.pdf"
    output_path = os.path.join(new_directory_path, output_file_name)
    
    output_doc.save(output_path)
    output_doc.close()

    print(f'Saved: {output_path}')

doc.close()

Saved: D:\Python Practice files\Individual pdf files for Transcripts\Bachelor_Degree_Transcript.pdf
Saved: D:\Python Practice files\Individual pdf files for Transcripts\Bachelor_Degree_Certificate.pdf
Saved: D:\Python Practice files\Individual pdf files for Transcripts\Info.pdf
Saved: D:\Python Practice files\Individual pdf files for Transcripts\Credential_Evaluation.pdf


In [ ]:
import csv
import re

keywords = ['LOAN-NO', 'FLD-COMM-NO', '1ST-PRIN-BAL', 'ESCROW-BAL', 'DUE-DATE', '1ST-DUE-DATE', 'FLOOD-ZONE']
with open('output_file.txt', 'r') as file:
    content = file.read()

pattern = "|".join([re.escape(keyword) for keyword in keywords])

matches = [(match.group(), match.start()) for match in re.finditer(pattern, content)]

data_rows = []
current_data = {keyword: "" for keyword in keywords}
processed_keywords = set()

for i in range(len(matches)-1):
    keyword, start_pos = matches[i]
    _, next_pos = matches[i+1]

    value = content[start_pos + len(keyword):next_pos].strip().split()[0]

    if keyword == "LOAN-NO":
        match = content[start_pos + len(keyword):next_pos].strip().split()[1]
        value = match if match else ""
    
    current_data[keyword] = value
    processed_keywords.add(keyword)

    if processed_keywords == set(keywords):
        data_rows.append(current_data.copy())
        current_data = {keyword: "" for keyword in keywords}
        processed_keywords.clear()

with open('output_file.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=keywords)
    writer.writeheader()
    writer.writerows(data_rows)
print("Data extraction and csv creation completed successfully")

